## Introduction du notebook

Nous sommes plusieurs petites équipes de Développeurs en IA, avec plusieurs projets proposés par notre instructeur !    


Notre équipe est chargée de créer un model IA capable de détecter les SPAM dans les Email, qu'elle puisse différencier ceux qui le sont avec ceux qui ne le sont pas, grâce à la méthode du NLP (Natural Langage processing)  

Le **NLP** est une technologie d’intelligence artificielle visant à permettre aux ordinateurs de comprendre le langage humain.
L’objectif de cette technologie est de permettre aux machines de lire, de déchiffrer, de comprendre et de donner sens au langage humain.  

Nous allons vous montrer une modèle qui possède une couche d'**Embedding**. Il s'agit d'un espace dit "peu dimensionnel" dans lequel nous pouvon traduire traduire des vecteurs à haute dimension. Cela facilite l'apprentissage automatique de notre IA sur des entrées de grande taille. L'avantage de ce modèle est qu'il pourra apprendre et être réutilisé dans plusieurs modèles (donc sur d'autres jeux de données).  

Pour notre projet nous avons à notre disposition une base de données comportant des mails, classé par 1 ou 0.
* 0 sont les mail non-spam.  
* 1 sont les mail considérés comme des spam.  

Dans ce notebook protocole/predict, vous pourrez suivre les étapes (Step by steps) pour pouvoir utiliser notre modèle avec différentes données.

Allons-y !

## Explication step by step

### Import the libraries that are recquired for our model

In [39]:
###### FIRST CELL TO EXECUTE : Importing libraries ######

# Basic imports
import pickle
import pandas as pd
import numpy as np

#preprocessing
from sklearn.model_selection import train_test_split

#modeling
import tensorflow as tf

#loading
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings(action="ignore")

### Import pickle, model and your dataset

In [40]:
###### SECOND CELL TO EXECUTE : Importing data that you wan to test ######

# Here, you need to replace the string 'PATHNAME_OF_YOUR_DATA_FILE' by the pathname of your file
# For example : model_df = pd.read_csv('data/eda_clean_df.csv')
# model_df = pd.read_csv('PATHNAME_OF_YOUR_DATA_FILE')

df = pd.read_csv('data/Spam Email raw text for NLP.csv')

In [41]:
# After you need Importing our trained model
# Here, you need to use the file "model.h5" that you will find here :
# https://github.com/Melodyellinn/Projet_NLP

model = keras.models.load_model('model.h5')

In [42]:
# ##### Run this function for creat sequences ##### #

def get_sequences(texts, tokenizer, train=True, max_seq_length=None):
    sequences = tokenizer.texts_to_sequences(texts)
    
    if train == True:
        max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
    
    sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_seq_length, padding='post')

    return sequences

In [43]:
# ##### Here, execute this cell for apply the presprocessing steps to clean data and creat the tokeniser for model. ##### #

# secure df
df = df.copy()

# Drop FILE_NAME column
df = df.drop('FILE_NAME', axis=1)
# Split df into X and y
y = df['CATEGORY']
X = df['MESSAGE']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
# Create tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=30000)
# Fit the tokenizer
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train = get_sequences(X_train, tokenizer, train=True)
X_test = get_sequences(X_test, tokenizer, train=False, max_seq_length=X_train.shape[1])

In [44]:
# ##### Execute this cell for use the model.evaluate your dataset with the IA model ##### #
results = model.evaluate(X_test, y_test, verbose=0)

In [46]:
# ##### Print the results ! ##### #
print("    Test Loss: {:.4f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))
print("     Test AUC: {:.4f}".format(results[2]))

    Test Loss: 0.0229
Test Accuracy: 99.19%
     Test AUC: 0.9989


In [48]:
# ##### result's predictions ##### #
my_prediction = model.predict(X_test)
print(my_prediction)

[[9.99998450e-01]
 [9.99937773e-01]
 [1.11707945e-04]
 ...
 [5.95420599e-04]
 [2.11740758e-06]
 [9.98863399e-01]]
